# LDC Implementation in Python/JAX
Written By: Sungje Park

In [1]:
import jax
from jax import numpy as jnp

import sys
sys.path.append('..')
from src.environment import *
from src.elements import *
from src.dynamics import *
from utils.test_utils import *
from src.methods import *
from tqdm import tqdm
import matplotlib.pyplot as plt

Defining Vars.

In [ ]:
rho_ref = 8 # reference density
Mu = .1 # kinematic Viscosity
Re = 100 # reynolds number
N_x = 100 # number of cells in the x dir
N_y = N_x # number of cells in the y dir
dx = 1 # spacing in x
dy = 1 # spacing in y
U_lid = Re*Mu/N_x # velocity of lid
Tau = 3*Mu+.5
dt = 1

In [3]:
cell_mesh = jnp.reshape(jnp.asarray(range(N_x*N_y)),(N_y,N_x))
node_mesh = jnp.reshape(jnp.asarray(range((N_x+1)*(N_y+1))),(N_y+1,N_x+1))
face_mesh = jnp.asarray(range(N_x*(N_y+1)+(N_x+1)*N_y))
vert_mesh = jnp.reshape(face_mesh[0:N_x*(N_y+1)],(N_y,N_x+1))
horz_mesh = jnp.reshape(face_mesh[N_x*(N_y+1):],(N_y+1,N_x))

In [4]:
dynamics = D2Q9(tau=Tau,delta_t = dt)
Element.dynamics = dynamics
Environment.dynamics = dynamics

In [5]:
def create_cell(cell_ind):
    loc = jnp.argwhere(cell_mesh == cell_ind)[0]
    x = loc[1]
    y = loc[0]
    pdf = dynamics.ones_pdf()
    face_ind = jnp.asarray([vert_mesh[y,x],
                            horz_mesh[y,x],
                            vert_mesh[y,x+1],
                            horz_mesh[y+1,x]])
    face_n = jnp.asarray([1,1,-1,-1])
    return Cell.pdf_init(pdf,face_ind,face_n)

cells = [create_cell(ind) for ind in tqdm(range(N_x*N_y))]

100%|██████████| 100/100 [00:01<00:00, 55.57it/s]


In [6]:
def create_node(node_ind):
    loc = jnp.argwhere(node_mesh==node_ind)[0]
    x = loc[1]
    y = loc[0]
    pdf = dynamics.ones_pdf()
    cell_index = []
    cell_dists = []
    cell_dist = jnp.sqrt(2)
    def cell_append(tl,tr,bl,br):
        if tl:
            cell_index.append(cell_mesh[y-1,x-1]) # top left
            cell_dists.append(cell_dist)
        if tr:
            cell_index.append(cell_mesh[y-1,x]) # top right
            cell_dists.append(cell_dist)

        if br:
            cell_index.append(cell_mesh[y,x]) # bottom right
            cell_dists.append(cell_dist)

        if bl:
            cell_index.append(cell_mesh[y,x-1])# bottom left
            cell_dists.append(cell_dist)

    if y == 0:
        type = 1
        vel = jnp.asarray([U_lid,0])
        if x!=0 and x!=N_x:
            cell_append(False,False,True,True)
        elif x==0:
            cell_append(False,False,False,True)
        elif x==N_x:
            cell_append(False,False,True,False)

    elif y == N_y:
        type = 1
        vel = jnp.asarray([0,0])
        if x!=0 and x!=N_x:
            cell_append(True,True,False,False)
        elif x==0:
            cell_append(False,True,False,False)
        elif x==N_x:
            cell_append(True,False,False,False)

    elif x == 0:
        type = 1
        vel = jnp.asarray([0,0])
        # since all corner cases handled
        cell_append(False,True,False,True)

    elif x == N_x:
        type = 1
        vel = jnp.asarray([0,0])
        cell_append(True,False,True,False)
    else:
        type = 0
        cell_append(True,True,True,True)

    node = Node.pdf_init(pdf,jnp.asarray(type),jnp.asarray(cell_index),jnp.asarray(cell_dists))

    if type == 1:
        node.vel = vel
    return node
nodes = [create_node(ind) for ind in tqdm(range((N_x+1)*(N_y+1)))]

100%|██████████| 121/121 [00:01<00:00, 62.79it/s]


In [7]:
node_vel = jnp.asarray([nodes[ind].vel for ind in tqdm(range((N_x+1)*(N_y+1)))])
node_vel = node_vel.reshape(N_y+1,N_x+1,2)


100%|██████████| 121/121 [00:00<00:00, 1231822.29it/s]


In [8]:
def create_face(face_ind):
    pdf = dynamics.ones_pdf()
    nodes_ind = []
    stencil_ind = []
    stencil_dists = jnp.asarray([.5,.5])
    
    if face_ind < N_x*(N_y+1): # vertical faces
        loc = jnp.argwhere(vert_mesh==face_ind)[0]
        x = loc[1]
        y = loc[0]
        nodes_ind.append(jnp.asarray([nodes[node_mesh[y,x]].type,node_mesh[y,x]]))
        nodes_ind.append(jnp.asarray([nodes[node_mesh[y+1,x]].type,node_mesh[y+1,x]]))

        if x == 0:
            stencil_ind.append(-1)
            stencil_ind.append(cell_mesh[y,x])
        elif x == N_x:
            stencil_ind.append(cell_mesh[y,x-1])
            stencil_ind.append(-1)
        else:
            stencil_ind.append(cell_mesh[y,x-1])
            stencil_ind.append(cell_mesh[y,x])

        n = jnp.asarray([1,0])
    else: # horizontal faces
        loc = jnp.argwhere(horz_mesh==face_ind)[0]
        x = loc[1]
        y = loc[0]
        nodes_ind.append(jnp.asarray([nodes[node_mesh[y,x]].type,node_mesh[y,x]]))
        nodes_ind.append(jnp.asarray([nodes[node_mesh[y,x+1]].type,node_mesh[y,x+1]]))

        if y == 0:
            stencil_ind.append(-1)
            stencil_ind.append(cell_mesh[y,x])
        elif y == N_y:
            stencil_ind.append(cell_mesh[y-1,x])
            stencil_ind.append(-1)
        else:
            stencil_ind.append(cell_mesh[y-1,x])
            stencil_ind.append(cell_mesh[y,x])

        n = jnp.asarray([0,1])

    face = Face.pdf_init(pdf,jnp.asarray(nodes_ind),jnp.asarray(stencil_ind),jnp.asarray(stencil_dists),n,jnp.asarray(1))
    return face

faces = [create_face(ind) for ind in tqdm(range((N_x)*(N_x+1)*2))]

100%|██████████| 220/220 [00:03<00:00, 62.71it/s]


In [9]:
env = Environment.create(cells,faces,nodes)
env.setMethods(Methods)
params,config = env.init()

In [10]:
params

{'cells': {'pdf': Array([[1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.

In [11]:
params["cells"]["pdf"][0]

Array([1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)

In [12]:
config["cells"]["faces_n"][0]

Array([ 1,  1, -1, -1], dtype=int32)

In [13]:
jnp.dot(params["faces"]["flux"][config["cells"]["faces_index"][1]].T,config["cells"]["faces_n"][1])

Array([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [14]:
jnp.dot(params["faces"]["flux"][config["cells"]["faces_index"][0]].T,config["cells"]["faces_n"][0])

Array([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [15]:
params["cells"]["pdf"]

Array([[1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1.

In [16]:
params["cells"]["pdf_eq"]

Array([[4.  , 1.  , 1.  , 1.  , 1.  , 0.25, 0.25, 0.25, 0.25],
       [4.  , 1.  , 1.  , 1.  , 1.  , 0.25, 0.25, 0.25, 0.25],
       [4.  , 1.  , 1.  , 1.  , 1.  , 0.25, 0.25, 0.25, 0.25],
       [4.  , 1.  , 1.  , 1.  , 1.  , 0.25, 0.25, 0.25, 0.25],
       [4.  , 1.  , 1.  , 1.  , 1.  , 0.25, 0.25, 0.25, 0.25],
       [4.  , 1.  , 1.  , 1.  , 1.  , 0.25, 0.25, 0.25, 0.25],
       [4.  , 1.  , 1.  , 1.  , 1.  , 0.25, 0.25, 0.25, 0.25],
       [4.  , 1.  , 1.  , 1.  , 1.  , 0.25, 0.25, 0.25, 0.25],
       [4.  , 1.  , 1.  , 1.  , 1.  , 0.25, 0.25, 0.25, 0.25],
       [4.  , 1.  , 1.  , 1.  , 1.  , 0.25, 0.25, 0.25, 0.25],
       [4.  , 1.  , 1.  , 1.  , 1.  , 0.25, 0.25, 0.25, 0.25],
       [4.  , 1.  , 1.  , 1.  , 1.  , 0.25, 0.25, 0.25, 0.25],
       [4.  , 1.  , 1.  , 1.  , 1.  , 0.25, 0.25, 0.25, 0.25],
       [4.  , 1.  , 1.  , 1.  , 1.  , 0.25, 0.25, 0.25, 0.25],
       [4.  , 1.  , 1.  , 1.  , 1.  , 0.25, 0.25, 0.25, 0.25],
       [4.  , 1.  , 1.  , 1.  , 1.  , 0.25, 0.25, 0.25,

In [17]:
env.step(params,config)
params

Traced<ShapedArray(float32[])>with<BatchTrace(level=1/0)> with
  val = Array([ 0.4949999 ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.40499997,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.   

{'cells': {'pdf': Array([[ 4.255     ,  0.5050001 ,  0.5050001 ,  0.5050001 ,  0.5050001 ,
          -0.4324999 , -0.4324999 , -0.4324999 , -0.4324999 ],
         [ 4.75      ,  1.        ,  1.        ,  1.        ,  1.        ,
           0.0625    ,  0.0625    ,  0.0625    ,  0.0625    ],
         [ 4.75      ,  1.        ,  1.        ,  1.        ,  1.        ,
           0.0625    ,  0.0625    ,  0.0625    ,  0.0625    ],
         [ 4.75      ,  1.        ,  1.        ,  1.        ,  1.        ,
           0.0625    ,  0.0625    ,  0.0625    ,  0.0625    ],
         [ 4.75      ,  1.        ,  1.        ,  1.        ,  1.        ,
           0.0625    ,  0.0625    ,  0.0625    ,  0.0625    ],
         [ 4.75      ,  1.        ,  1.        ,  1.        ,  1.        ,
           0.0625    ,  0.0625    ,  0.0625    ,  0.0625    ],
         [ 4.75      ,  1.        ,  1.        ,  1.        ,  1.        ,
           0.0625    ,  0.0625    ,  0.0625    ,  0.0625    ],
         [ 4.75   

In [18]:
env.methods.calc_cell_macros(params,config)

{'cells': {'pdf': Array([[ 4.255     ,  0.5050001 ,  0.5050001 ,  0.5050001 ,  0.5050001 ,
          -0.4324999 , -0.4324999 , -0.4324999 , -0.4324999 ],
         [ 4.75      ,  1.        ,  1.        ,  1.        ,  1.        ,
           0.0625    ,  0.0625    ,  0.0625    ,  0.0625    ],
         [ 4.75      ,  1.        ,  1.        ,  1.        ,  1.        ,
           0.0625    ,  0.0625    ,  0.0625    ,  0.0625    ],
         [ 4.75      ,  1.        ,  1.        ,  1.        ,  1.        ,
           0.0625    ,  0.0625    ,  0.0625    ,  0.0625    ],
         [ 4.75      ,  1.        ,  1.        ,  1.        ,  1.        ,
           0.0625    ,  0.0625    ,  0.0625    ,  0.0625    ],
         [ 4.75      ,  1.        ,  1.        ,  1.        ,  1.        ,
           0.0625    ,  0.0625    ,  0.0625    ,  0.0625    ],
         [ 4.75      ,  1.        ,  1.        ,  1.        ,  1.        ,
           0.0625    ,  0.0625    ,  0.0625    ,  0.0625    ],
         [ 4.75   

In [19]:
env.step(params,config)
params["faces"]["pdf"]

Traced<ShapedArray(float32[])>with<BatchTrace(level=1/0)> with
  val = Array([ 4.704974 , -2.2274995,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  1.8225001, -4.8215237,
       -2.2274995,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  1.2149999,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.

Array([[ 4.3636    ,  0.9191501 ,  0.62496257, ..., -0.4324999 ,
        -0.4324999 , -0.14333743],
       [ 4.5025    ,  0.5050001 ,  0.75250006, ...,  0.0625    ,
         0.0625    , -0.4324999 ],
       [ 4.75      ,  1.        ,  1.        , ...,  0.0625    ,
         0.0625    ,  0.0625    ],
       ...,
       [ 4.75      ,  1.        ,  1.        , ...,  0.0625    ,
         0.0625    ,  0.0625    ],
       [ 4.75      ,  1.        ,  1.        , ...,  0.0625    ,
         0.0625    ,  0.0625    ],
       [ 4.75      ,  1.        ,  1.        , ...,  0.0625    ,
         0.0625    ,  0.0625    ]], dtype=float32)

In [20]:
for i in tqdm(range(10)):
    env.step(params,config)

  0%|          | 0/10 [00:00<?, ?it/s]

Traced<ShapedArray(float32[])>with<BatchTrace(level=1/0)> with
  val = Array([ 61.98391  , -48.7377   ,  10.023748 ,   0.       ,   0.       ,
         0.       ,   0.       ,  -8.20125  ,  44.250286 , -66.58868  ,
       -52.07895  ,  13.364996 ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,  -9.1125   ,  33.145195 ,
         6.6824985,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,  -3.6449995,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.  

 40%|████      | 4/10 [00:00<00:00, 32.05it/s]

Traced<ShapedArray(float32[])>with<BatchTrace(level=1/0)> with
  val = Array([ 3.44686469e+05, -4.00036906e+05,  2.18777750e+05, -6.86011875e+04,
        1.27242227e+04, -1.11112686e+04,  6.06293555e+04, -1.99800938e+05,
        3.75920469e+05, -3.38252844e+05, -4.38595156e+05,  2.93600656e+05,
       -1.07001266e+05,  2.04952891e+04, -1.64640051e+03,  1.29784778e+03,
       -1.64653379e+04,  8.65752891e+04, -2.35480438e+05,  3.46730281e+05,
        1.90493328e+05, -9.71823672e+04,  2.35176797e+04, -2.21023633e+03,
        0.00000000e+00,  0.00000000e+00,  1.61154553e+03, -1.70123809e+04,
        6.86328438e+04, -1.29968336e+05, -3.98322930e+04,  1.50067529e+04,
       -1.89448828e+03,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.25479126e+03, -9.65524902e+03,  2.46535039e+04,
        4.18020312e+03, -9.02137329e+02,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        5.41282471e+02, -2.40

100%|██████████| 10/10 [00:00<00:00, 32.28it/s]

Traced<ShapedArray(float32[])>with<BatchTrace(level=1/0)> with
  val = Array([ 6.1370876e+10, -8.3269992e+10,  6.3126503e+10, -3.3614838e+10,
        1.5720473e+10, -1.4779653e+10,  3.0367519e+10, -5.6987386e+10,
        7.5477574e+10, -5.6821182e+10, -8.8008909e+10,  7.8068711e+10,
       -4.7596823e+10,  2.0528191e+10, -7.3458447e+09,  6.6545741e+09,
       -1.7654149e+10,  4.0645718e+10, -6.6433323e+10,  7.6811207e+10,
        6.0792230e+10, -4.5529223e+10,  2.3527387e+10, -8.3604035e+09,
        2.2693619e+09, -1.9443304e+09,  6.7252582e+09, -1.8673302e+10,
        3.5616956e+10, -4.7265927e+10, -2.5082024e+10,  1.6988707e+10,
       -7.5551099e+09,  2.1870861e+09, -4.3206938e+08,  3.4628208e+08,
       -1.6466868e+09,  5.5941212e+09, -1.2336974e+10,  1.7952559e+10,
        6.8028472e+09, -4.1898591e+09,  1.5733432e+09, -3.5361920e+08,
        4.5819632e+07, -3.4071224e+07,  2.5095144e+08, -1.0962894e+09,
        2.8570893e+09, -4.5589837e+09, -1.2536047e+09,  6.8735770e+08,
      

In [21]:
params

{'cells': {'pdf': Array([[-3.50314436e+13, -2.99146821e+13, -2.99146821e+13,
          -2.99146821e+13, -2.99146821e+13, -2.86354907e+13,
          -2.86354907e+13, -2.86354907e+13, -2.86354907e+13],
         [ 4.93781485e+13,  4.22825941e+13,  4.22825941e+13,
           4.22825941e+13,  4.22825941e+13,  4.05087097e+13,
           4.05087097e+13,  4.05087097e+13,  4.05087097e+13],
         [-4.05302810e+13, -3.49025526e+13, -3.49025526e+13,
          -3.49025526e+13, -3.49025526e+13, -3.34956195e+13,
          -3.34956195e+13, -3.34956195e+13, -3.34956195e+13],
         [ 2.45312468e+13,  2.13090587e+13,  2.13090587e+13,
           2.13090587e+13,  2.13090587e+13,  2.05035112e+13,
           2.05035112e+13,  2.05035112e+13,  2.05035112e+13],
         [-1.37851201e+13, -1.21095565e+13, -1.21095565e+13,
          -1.21095565e+13, -1.21095565e+13, -1.16906651e+13,
          -1.16906651e+13, -1.16906651e+13, -1.16906651e+13],
         [ 1.30566555e+13,  1.14742652e+13,  1.14742652e+13,
   